# Solar Thermal Collector Module

Models flat plate solar thermal collector (FPSC) for direct thermal energy capture. Uses Hottel-Whillier-Bliss equation for useful heat output.

## Energy Analysis

**Useful heat output (Hottel-Whillier-Bliss):**
$$\dot{Q}_{collector} = A_{coll} F_R \left[G(\tau\alpha) - U_L(T_{inlet} - T_{amb})\right]$$

**Overall heat loss coefficient:**
$$U_L = U_t + U_b$$

**Wind convection coefficient:**
$$h_w = 5.7 + 3.8 V_{wind}$$

**Energy efficiency:**
$$\eta_{coll} = \frac{\dot{Q}_{collector}}{A_{coll} \cdot G} = F_R\left[(\tau\alpha) - U_L\frac{T_{inlet} - T_{amb}}{G}\right]$$

Typical flat plate: $\eta_{en} = 40-60\%$

## Exergy Analysis

**Solar exergy input:**
$$\dot{Ex}_{in,rad} = A_{coll} G \left(1 - \frac{T_{amb}}{T_{sun}}\right)$$

**Fluid exergy input:**
$$\dot{Ex}_{in,fluid} = \dot{m} c_p \left[(T_{inlet} - T_{amb}) - T_{amb}\ln\frac{T_{inlet}}{T_{amb}}\right]$$

**Fluid exergy output:**
$$\dot{Ex}_{out,fluid} = \dot{m} c_p \left[(T_{outlet} - T_{amb}) - T_{amb}\ln\frac{T_{outlet}}{T_{amb}}\right]$$

**Exergy destruction:**
$$\dot{Ex}_{d,coll} = \dot{Ex}_{in,rad} + \dot{Ex}_{in,fluid} - \dot{Ex}_{out,fluid}$$

**Exergy efficiency:**
$$\Psi_{coll} = \frac{\dot{Ex}_{out,fluid} - \dot{Ex}_{in,fluid}}{A_{coll} G \left(1 - \frac{T_{amb}}{T_{sun}}\right)}$$

Typical flat plate: $\Psi = 15-25\%$

## Parameters

In [1]:
# Solar Thermal Collector Default Parameters
const STC_DEFAULTS = (
    F_R = 0.80,             # Heat removal factor [-]
    τα = 0.82,              # Transmittance-absorptance product [-]
    U_L = 4.5,              # Overall heat loss coefficient [W/(m²·K)]
    T_sun = 5777.0,         # Effective sun temperature [K]
    c_p_fluid = 4186.0      # Fluid specific heat (water) [J/(kg·K)]
)

(F_R = 0.8, τα = 0.82, U_L = 4.5, T_sun = 5777.0, c_p_fluid = 4186.0)

## Core Functions

In [2]:
"""
    SolarThermalCollector

Struct holding flat plate solar thermal collector parameters.
"""
struct SolarThermalCollector
    A::Float64              # Collector area [m²]
    F_R::Float64            # Heat removal factor [-]
    τα::Float64             # Transmittance-absorptance product [-]
    U_L::Float64            # Overall heat loss coefficient [W/(m²·K)]
    T_sun::Float64          # Sun temperature [K]
    c_p::Float64            # Fluid specific heat [J/(kg·K)]
    
    function SolarThermalCollector(A::Float64;
            F_R::Float64 = 0.80,
            τα::Float64 = 0.82,
            U_L::Float64 = 4.5,
            T_sun::Float64 = 5777.0,
            c_p::Float64 = 4186.0)
        new(A, F_R, τα, U_L, T_sun, c_p)
    end
end

SolarThermalCollector

In [3]:
"""
    solar_exergy_factor(T_amb::Float64, T_sun::Float64) -> Float64

Calculate Petela-Landsberg-Press solar exergy factor.
"""
function solar_exergy_factor(T_amb::Float64, T_sun::Float64)
    ratio = T_amb / T_sun
    return 1.0 - (4.0/3.0) * ratio + (1.0/3.0) * ratio^4
end

solar_exergy_factor

In [4]:
"""
    useful_heat(coll::SolarThermalCollector, G::Float64, T_inlet::Float64, T_amb::Float64) -> Float64

Calculate useful heat output [W] using Hottel-Whillier-Bliss equation.
G: irradiance [W/m²], T_inlet: fluid inlet temp [K], T_amb: ambient temp [K]
"""
function useful_heat(coll::SolarThermalCollector, G::Float64, T_inlet::Float64, T_amb::Float64)
    if G <= 0
        return 0.0
    end
    Q = coll.A * coll.F_R * (G * coll.τα - coll.U_L * (T_inlet - T_amb))
    return max(Q, 0.0)  # Cannot be negative
end

useful_heat

In [5]:
"""
    outlet_temperature(coll::SolarThermalCollector, G::Float64, T_inlet::Float64, 
                       T_amb::Float64, m_dot::Float64) -> Float64

Calculate fluid outlet temperature [K].
m_dot: mass flow rate [kg/s]
"""
function outlet_temperature(coll::SolarThermalCollector, G::Float64, T_inlet::Float64, 
                            T_amb::Float64, m_dot::Float64)
    Q = useful_heat(coll, G, T_inlet, T_amb)
    if m_dot <= 0 || Q <= 0
        return T_inlet
    end
    ΔT = Q / (m_dot * coll.c_p)
    return T_inlet + ΔT
end

outlet_temperature

In [6]:
"""
    collector_efficiency(coll::SolarThermalCollector, G::Float64, T_inlet::Float64, T_amb::Float64) -> Float64

Calculate collector energy efficiency [-].
"""
function collector_efficiency(coll::SolarThermalCollector, G::Float64, T_inlet::Float64, T_amb::Float64)
    if G <= 0
        return 0.0
    end
    Q = useful_heat(coll, G, T_inlet, T_amb)
    return Q / (coll.A * G)
end

collector_efficiency

In [7]:
"""
    stagnation_temperature(coll::SolarThermalCollector, G::Float64, T_amb::Float64) -> Float64

Calculate stagnation temperature [K] (no flow condition).
"""
function stagnation_temperature(coll::SolarThermalCollector, G::Float64, T_amb::Float64)
    if G <= 0
        return T_amb
    end
    # At stagnation: Q = 0, so G*τα = U_L*(T_stag - T_amb)
    return T_amb + (G * coll.τα) / coll.U_L
end

stagnation_temperature

In [8]:
"""
    fluid_exergy_rate(T::Float64, T_0::Float64, m_dot::Float64, c_p::Float64) -> Float64

Calculate fluid exergy rate [W] at temperature T.
"""
function fluid_exergy_rate(T::Float64, T_0::Float64, m_dot::Float64, c_p::Float64)
    if T <= T_0 || m_dot <= 0
        return 0.0
    end
    return m_dot * c_p * ((T - T_0) - T_0 * log(T / T_0))
end

fluid_exergy_rate

In [9]:
"""
    analyze_collector(coll::SolarThermalCollector, G::Float64, T_inlet::Float64, 
                      T_amb::Float64, m_dot::Float64) -> NamedTuple

Complete energy and exergy analysis.
G: irradiance [W/m²], T_inlet: inlet temp [K], T_amb: ambient [K], m_dot: flow [kg/s]
"""
function analyze_collector(coll::SolarThermalCollector, G::Float64, T_inlet::Float64, 
                           T_amb::Float64, m_dot::Float64)
    # Energy analysis
    E_solar_in = coll.A * G
    Q_useful = useful_heat(coll, G, T_inlet, T_amb)
    T_outlet = outlet_temperature(coll, G, T_inlet, T_amb, m_dot)
    η_en = collector_efficiency(coll, G, T_inlet, T_amb)
    Q_loss = E_solar_in - Q_useful
    
    # Exergy analysis
    T_0 = T_amb  # Dead state = ambient
    ψ_solar = solar_exergy_factor(T_amb, coll.T_sun)
    Ex_solar_in = E_solar_in * ψ_solar
    
    Ex_fluid_in = fluid_exergy_rate(T_inlet, T_0, m_dot, coll.c_p)
    Ex_fluid_out = fluid_exergy_rate(T_outlet, T_0, m_dot, coll.c_p)
    
    # Useful exergy gain
    Ex_gain = Ex_fluid_out - Ex_fluid_in
    
    # Exergy destruction
    Ex_d = Ex_solar_in - Ex_gain
    Ex_d = max(Ex_d, 0.0)
    
    # Exergy efficiency
    Ψ = Ex_solar_in > 0 ? Ex_gain / Ex_solar_in : 0.0
    
    return (
        # Conditions
        G = G,
        T_inlet = T_inlet,
        T_outlet = T_outlet,
        T_amb = T_amb,
        m_dot = m_dot,
        # Energy [W]
        E_solar_in = E_solar_in,
        Q_useful = Q_useful,
        Q_loss = Q_loss,
        η_en = η_en,
        # Exergy [W]
        Ex_solar_in = Ex_solar_in,
        Ex_fluid_in = Ex_fluid_in,
        Ex_fluid_out = Ex_fluid_out,
        Ex_gain = Ex_gain,
        Ex_d = Ex_d,
        ψ_solar = ψ_solar,
        Ψ = Ψ
    )
end

analyze_collector

## Tests

In [10]:
using Test

function run_stc_tests()
    println("Running Solar Thermal Collector Tests...")
    
    # Create test collector: 4 m²
    coll = SolarThermalCollector(4.0, F_R=0.80, τα=0.82, U_L=4.5)
    
    # Test 1: Parameters
    @testset "Parameters" begin
        @test coll.A == 4.0
        @test coll.F_R == 0.80
        @test coll.τα == 0.82
    end
    
    # Test 2: Solar exergy factor
    @testset "Solar Exergy Factor" begin
        ψ = solar_exergy_factor(298.0, 5777.0)
        @test ψ ≈ 0.93 atol=0.01
    end
    
    # Test 3: Useful heat
    @testset "Useful Heat" begin
        G = 1000.0
        T_inlet = 313.15  # 40°C
        T_amb = 298.15    # 25°C
        Q = useful_heat(coll, G, T_inlet, T_amb)
        @test Q > 0
        @test Q < coll.A * G  # Less than incident
    end
    
    # Test 4: Zero irradiance
    @testset "Zero Irradiance" begin
        @test useful_heat(coll, 0.0, 313.15, 298.15) == 0.0
        @test collector_efficiency(coll, 0.0, 313.15, 298.15) == 0.0
    end
    
    # Test 5: Outlet temperature
    @testset "Outlet Temperature" begin
        T_out = outlet_temperature(coll, 1000.0, 313.15, 298.15, 0.05)
        @test T_out > 313.15  # Outlet warmer than inlet
    end
    
    # Test 6: Stagnation temperature
    @testset "Stagnation" begin
        T_stag = stagnation_temperature(coll, 1000.0, 298.15)
        @test T_stag > 298.15 + 100  # Should be quite hot
        # At stagnation, useful heat should be ~0
        Q_stag = useful_heat(coll, 1000.0, T_stag, 298.15)
        @test Q_stag ≈ 0.0 atol=1.0
    end
    
    # Test 7: Efficiency curve
    @testset "Efficiency Curve" begin
        G = 1000.0
        T_amb = 298.15
        # Higher inlet temp = lower efficiency
        η_low = collector_efficiency(coll, G, 303.15, T_amb)  # 30°C inlet
        η_high = collector_efficiency(coll, G, 343.15, T_amb)  # 70°C inlet
        @test η_low > η_high
    end
    
    # Test 8: Exergy analysis
    @testset "Exergy Analysis" begin
        result = analyze_collector(coll, 1000.0, 313.15, 298.15, 0.05)
        
        # Exergy efficiency < energy efficiency
        @test result.Ψ < result.η_en
        # Exergy destruction positive
        @test result.Ex_d > 0
        # Exergy gain positive
        @test result.Ex_gain > 0
    end
    
    println("All tests passed!")
end

run_stc_tests()

Running Solar Thermal Collector Tests...
Test Summary: | Pass  Total  Time
Parameters    |    3      3  0.0s
Test Summary:       | Pass  Total  Time
Solar Exergy Factor |    1      1  0.0s
Test Summary: | Pass  Total  Time
Useful Heat   |    2      2  0.0s
Test Summary:   | Pass  Total  Time
Zero Irradiance |    2      2  0.0s
Test Summary:      | Pass  Total  Time
Outlet Temperature |    1      1  0.0s
Test Summary: | Pass  Total  Time
Stagnation    |    2      2  0.0s
Test Summary:    | Pass  Total  Time
Efficiency Curve |    1      1  0.0s
Test Summary:   | Pass  Total  Time
Exergy Analysis |    3      3  0.0s
All tests passed!


## Example Usage

In [11]:
# Example: 4 m² flat plate collector
coll = SolarThermalCollector(4.0)

result = analyze_collector(coll, 800.0, 313.15, 298.15, 0.04)

println("Solar Thermal Collector Analysis")
println("="^50)
println("Collector area:     $(coll.A) m²")
println("Irradiance:         $(result.G) W/m²")
println("Inlet temp:         $(round(result.T_inlet - 273.15, digits=1)) °C")
println("Outlet temp:        $(round(result.T_outlet - 273.15, digits=1)) °C")
println("Ambient temp:       $(round(result.T_amb - 273.15, digits=1)) °C")
println("Flow rate:          $(result.m_dot*3600) kg/h")
println("─"^50)
println("Solar energy in:    $(round(result.E_solar_in, digits=0)) W")
println("Useful heat:        $(round(result.Q_useful, digits=0)) W")
println("Heat loss:          $(round(result.Q_loss, digits=0)) W")
println("Energy efficiency:  $(round(result.η_en*100, digits=1))%")
println("─"^50)
println("Solar exergy in:    $(round(result.Ex_solar_in, digits=0)) W")
println("Exergy gain:        $(round(result.Ex_gain, digits=0)) W")
println("Exergy destruction: $(round(result.Ex_d, digits=0)) W")
println("Exergy efficiency:  $(round(result.Ψ*100, digits=1))%")

Solar Thermal Collector Analysis
Collector area:     4.0 m²
Irradiance:         800.0 W/m²
Inlet temp:         40.0 °C
Outlet temp:        51.2 °C
Ambient temp:       25.0 °C
Flow rate:          144.0 kg/h
──────────────────────────────────────────────────
Solar energy in:    3200.0 W
Useful heat:        1883.0 W
Heat loss:          1317.0 W
Energy efficiency:  58.8%
──────────────────────────────────────────────────
Solar exergy in:    2980.0 W
Exergy gain:        122.0 W
Exergy destruction: 2858.0 W
Exergy efficiency:  4.1%


In [ ]:
# Efficiency curve vs inlet temperature (wrapped in let block for NBInclude compatibility)
let
    println("\nEfficiency vs Inlet Temperature (G=1000 W/m²)")
    println("="^55)
    println("T_in[°C]   T_out[°C]   η_en[%]   Ψ[%]    Q[W]")
    println("─"^55)

    G = 1000.0
    T_amb = 298.15

    for T_in_C in [30, 40, 50, 60, 70, 80]
        T_inlet = T_in_C + 273.15
        result = analyze_collector(coll, G, T_inlet, T_amb, 0.05)
        println("$(lpad(T_in_C, 6))     $(lpad(round(result.T_outlet - 273.15, digits=1), 8))   $(lpad(round(result.η_en*100, digits=1), 7))   $(lpad(round(result.Ψ*100, digits=1), 5))   $(lpad(round(result.Q_useful, digits=0), 6))")
    end
end

## Export

Functions available for import:
- `SolarThermalCollector` - Model struct
- `solar_exergy_factor` - Petela-Landsberg-Press factor
- `useful_heat` - Hottel-Whillier-Bliss equation
- `outlet_temperature` - Fluid outlet temperature
- `collector_efficiency` - Energy efficiency
- `stagnation_temperature` - No-flow temperature
- `fluid_exergy_rate` - Fluid exergy at given T
- `analyze_collector` - Complete analysis